In [9]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [1]:
import os

cat = '3848'
inference_image_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848/3848"
focus_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848/focus_3848"
additional_focus_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3848/additional_focus"
yes_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/3848/fewshot_additional_focus/yes"
no_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/3848/fewshot_additional_focus/no"
detect_path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/3848/fewshot_additional_focus/detect"

config = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/additional_focus/3848/attention-rpn_r50_c4_voc-split1_5shot-fine-tuning.py"
checkpoint = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/additional_focus/3848/iter_1200.pth"
support_images_dir = '/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/demo/demo_detection_images/support_images'

In [2]:
import cv2

def get_max_object(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    ## (2) Morph-op to remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

    ## (3) Find the max-area contour
    cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    x1 = []
    y1 = []
    x2 = []
    y2 = []
    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
        x1.append(x)
        y1.append(y)
        x2.append(x+w)
        y2.append(y+h)
    x1 = min(x1)
    y1 = min(y1)
    x2 = max(x2)
    y2 = max(y2)
    return x1, y1, x2, y2

In [3]:
from keras.applications.resnet import ResNet152, preprocess_input
import keras.utils as image
import numpy as np
import os
import time
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
warnings.filterwarnings("ignore")
model_embedding = ResNet152(include_top=False, weights='imagenet', pooling='avg')
def return_image_embedding(img):
    if isinstance(img, str):
        img = image.load_img(img, target_size=(224, 224))
    else:
        img = img.resize((224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model_embedding.predict(x, verbose=False)
    preds = preds[0].T
    return preds

2023-05-22 16:25:44.302064: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-22 16:25:44.327398: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 16:25:44.691860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-22 16:25:45.107776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

In [4]:
from torchvision.io import read_image
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torchvision.models.feature_extraction import create_feature_extractor

weights = ViT_B_16_Weights.DEFAULT
model_vit = vit_b_16(weights=weights)
model_vit.eval()

return_nodes = {
    "getitem_5": "output"
}
model_vit = create_feature_extractor(model_vit, return_nodes=return_nodes)
preprocess = weights.transforms()

def return_image_embedding_VIT(img):
    if isinstance(img, str):
        img = read_image(img)
    batch = preprocess(img).unsqueeze(0)
    preds = model_vit(batch)['output'].squeeze(0).detach().numpy()
    return preds

### Embedding only

In [19]:
from scipy.spatial import distance
import cv2
import os
import shutil
from tqdm import tqdm

THR = 0.55

focus_embedding = []
for img_focus in os.listdir(focus_path):
  img1 = os.path.join(focus_path, img_focus)
  x1, y1, x2, y2 = get_max_object(img1)
  img_embed = return_image_embedding(model_embedding,img1)
  focus_embedding.append(img_embed)

for img_name in tqdm(os.listdir(inference_image_path)):
  img = os.path.join(inference_image_path, img_name)
  img_embed = return_image_embedding(model_embedding, img)
  scr = [round(1- distance.cosine(img_embed,x), 2) for x in focus_embedding]
  if any(x >= THR for x in scr):
    shutil.copy(os.path.join(inference_image_path, img_name), os.path.join("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/door_compare_focus_background/embedding_only/yes", img_name))
    # continue
  else:
    shutil.copy(os.path.join(inference_image_path, img_name), os.path.join("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/door_compare_focus_background/embedding_only/no", img_name))
    # continue
# img = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/test_img.png"
# img_embed = return_image_embedding(model_embedding,img)
# scr = [round(1- distance.cosine(img_embed,x), 2) for x in focus_embedding]
# print(scr)
# print(sum(scr) / len(scr))


100%|██████████| 2616/2616 [01:51<00:00, 23.51it/s]


### Object Detection + Embedding

In [5]:
# Copyright (c) OpenMMLab. All rights reserved.
"""Inference Attention RPN Detector with support instances.
"""  # nowq

import os
from tqdm import tqdm
from argparse import ArgumentParser

from mmdet.apis import show_result_pyplot

from mmfewshot.detection.apis import (inference_detector, init_detector,
                                    process_support_images)
import cv2
import numpy as np
import torch
import torchvision
import random
from scipy.spatial import distance
import keras.utils as image
import shutil
from mmdet.apis import show_result_pyplot

def parse_args():
    parser = ArgumentParser('attention rpn inference.')
    args = parser.parse_args()
    return args

def nms(boxes, thr):
  boxbox = torch.from_numpy(boxes[:, 0:4])
  score = torch.from_numpy(boxes[:, 4])
  result = torchvision.ops.nms(boxbox, score, thr)
  return result.numpy()

def get_ebedd_origin(model_embedding, image):
    cropped_embed = return_image_embedding(model_embedding, image)
    scr = [round(1- distance.cosine(cropped_embed, x), 2) for x in focus_embedding]
    print(scr)
    print(sum(scr) / len(scr))

# build the model from a config file and a checkpoint file
include = []
exclude = []

device = 'cuda:0'
score_thr = 0.1
SIMILAR_THR = 0.55
model = init_detector(config, checkpoint, device=device)
print(next(model.parameters()).device)
print(next(model.parameters()).is_cuda)

# prepare support images, each demo image only contain one instance
files = os.listdir(support_images_dir)
support_images = [
    os.path.join(support_images_dir, file) for file in files
]

classes = [file.split('.')[0] for file in files]
support_labels = [[file.split('.')[0]] for file in files]
process_support_images(model, support_images, support_labels, classes=classes)

focus_embedding = []
print("FOCUS")
for img_focus in os.listdir(focus_path):
  print(img_focus)
  img_path = os.path.join(focus_path, img_focus)
  img = image.load_img(img_path, target_size=(224, 224))
  img_embed = return_image_embedding(img)
  focus_embedding.append(img_embed)
  
additional_embedding = []
additional_crop_embedding = []
print("ADDITIONAL FOCUS")
for img_focus in os.listdir(additional_focus_path):
  print(img_focus)
  img_path = os.path.join(additional_focus_path, img_focus)
  additional_embedding.append(return_image_embedding(img_path))
  x1, y1, x2, y2 = get_max_object(img_path)
  img = image.load_img(img_path)
  img_cropped = img.crop((x1, y1, x2, y2))
  crop_embedd = return_image_embedding(img_cropped)
  additional_crop_embedding.append(crop_embedd)

for image_name in tqdm(os.listdir(inference_image_path)):
# for image_name in ["B00AIIFR2U.jpg"]:
    image_path = os.path.join(inference_image_path, image_name)
    img_embed = return_image_embedding(image_path)
    scr = [round(1- distance.cosine(img_embed,x), 2) for x in focus_embedding + additional_embedding]
    # print(scr)
    if any(x >= SIMILAR_THR for x in scr):
        shutil.copy(os.path.join(inference_image_path, image_name), os.path.join(yes_path, image_name))
        include.append(image_name)
    else:
        try:
            result = inference_detector(model, image_path)
        except:
            print(image_path)
            continue
        res = sorted([x for x in result[0] if x[4] >= score_thr], key=lambda x: x[4], reverse=True)
        if len(res) > 0:
            list_scr = []
            for bbox in res[0:1]:
                img = image.load_img(image_path)
                cropped = img.crop((bbox[0], bbox[1], bbox[2], bbox[3]))
                # cropped.save('./test_img.png')
                # result = inference_detector(model, './test_img.png')
                # print("cropped: ", *result)
                # show_result_pyplot(model, './test_img.png', result, score_thr=score_thr)
                cropped_embed = return_image_embedding(cropped)
                scr = [round(1- distance.cosine(cropped_embed, x), 2) for x in additional_crop_embedding]
                # scr2 = [round(1- distance.cosine(cropped_embed, x), 2) for x in focus_embedding]
                list_scr.append(max(scr))
                # print(scr)
                # print(scr2)
            if max(list_scr) >= SIMILAR_THR:
                shutil.copy(os.path.join(inference_image_path, image_name), os.path.join(yes_path, image_name))
                output_path = os.path.join(detect_path, image_name)
                custom_res = np.expand_dims(np.expand_dims(np.array(res[0:1][0]), 0), 0)
                model.show_result(image_path, custom_res, out_file=output_path, score_thr=score_thr)
                include.append(image_name)
            else:
                shutil.copy(os.path.join(inference_image_path, image_name), os.path.join(no_path, image_name))
                exclude.append(image_name)
        else:
            shutil.copy(os.path.join(inference_image_path, image_name), os.path.join(no_path, image_name))
            exclude.append(image_name)

load checkpoint from local path: /home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/additional_focus/3848/iter_1200.pth
cuda:0
True
FOCUS
B09LHC752N.jpg


2023-05-22 16:25:55.490141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-22 16:25:55.537962: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


B07PDQFDMF.jpg
B07WFMJ84J.jpg
B0734MYZPC.jpg
B07WN1874X.jpg
ADDITIONAL FOCUS
B0B4SJ29N6.jpg
B0B2V6QR96.jpg
B0BB8Z6585.jpg
B0B8PGM7H4.jpg
B0747L2VLZ.jpg


100%|██████████| 8027/8027 [07:32<00:00, 17.74it/s]


In [6]:
print(len(os.listdir(inference_image_path)))
print(len(os.listdir(yes_path)))
print(len(os.listdir(no_path)))
print(len(os.listdir(detect_path)))

8027
7477
550
593


### Object Detection + Embedding Version 2

In [ ]:
# from scipy.spatial import distance

# from torchvision.io import read_image
# from torchvision.models import resnet50, ResNet50_Weights, vit_b_16, ViT_B_16_Weights
# import torch.nn as nn
# from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names

# img = read_image("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/img/focus_door/no_background/B0B8VT3225.jpg")
# print(img)
# # Step 1: Initialize model with the best available weights
# weights = ViT_B_16_Weights.DEFAULT
# model_vit = vit_b_16(weights=weights)
# model_vit.eval()
# # train_nodes, eval_nodes = get_graph_node_names(model_vit)
# # print(eval_nodes)
# # print(model_vit)

# return_nodes = {
#     "getitem_5": "output"
# }

# model_vit = create_feature_extractor(model_vit, return_nodes=return_nodes)

# # Step 2: Initialize the inference transforms
# preprocess = weights.transforms()

# # Step 3: Apply inference preprocessing transforms
# batch = preprocess(img).unsqueeze(0)

# # Step 4: Use the model_vit and print the predicted category
# prediction = model_vit(batch)['output'].squeeze(0).detach().numpy()
# # print(prediction.shape)
# print(round(1- distance.cosine(prediction,prediction), 2))

In [31]:
from torchvision.io import read_image
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torchvision.models.feature_extraction import create_feature_extractor

weights = ViT_B_16_Weights.DEFAULT
model_vit = vit_b_16(weights=weights)
model_vit.eval()

return_nodes = {
    "getitem_5": "output"
}
model_vit = create_feature_extractor(model_vit, return_nodes=return_nodes)
preprocess = weights.transforms()

def return_image_cropped_embedding_VIT(img, x1=None, y1=None, x2=None, y2=None):
    if isinstance(img, str):
        img = read_image(img)
    if x1:
        img = img[y1:y2, x1:x2]
    batch = preprocess(img).unsqueeze(0)
    preds = model_vit(batch)['output'].squeeze(0).detach().numpy()
    return preds

768


In [27]:
# Copyright (c) OpenMMLab. All rights reserved.
"""Inference Attention RPN Detector with support instances.
"""  # nowq

import os
from tqdm import tqdm
from argparse import ArgumentParser

from mmdet.apis import show_result_pyplot

from mmfewshot.detection.apis import (inference_detector, init_detector,
                                    process_support_images)
import cv2
import numpy as np
import torch
import torchvision
import random
from scipy.spatial import distance
import keras.utils as image
import shutil
from mmdet.apis import show_result_pyplot

def parse_args():
    parser = ArgumentParser('attention rpn inference.')
    args = parser.parse_args()
    return args

def nms(boxes, thr):
  boxbox = torch.from_numpy(boxes[:, 0:4])
  score = torch.from_numpy(boxes[:, 4])
  result = torchvision.ops.nms(boxbox, score, thr)
  return result.numpy()

def get_ebedd_origin(model_embedding, image):
    cropped_embed = return_image_embedding(model_embedding, image)
    scr = [round(1- distance.cosine(cropped_embed, x), 2) for x in focus_embedding]
    print(scr)
    print(sum(scr) / len(scr))

# build the model from a config file and a checkpoint file
include = []
exclude = []

device = 'cuda:0'
score_thr = 0.2
SIMILAR_THR = 0.55
CROPPED_THR = 0.61
THR_MAX_KNN = 0.7
THR_MIN_KNN = 0.4

model = init_detector(config, checkpoint, device=device)
print(next(model.parameters()).device)
print(next(model.parameters()).is_cuda)

# prepare support images, each demo image only contain one instance
files = os.listdir(support_images_dir)
support_images = [
    os.path.join(support_images_dir, file) for file in files
]

classes = [file.split('.')[0] for file in files]
support_labels = [[file.split('.')[0]] for file in files]
process_support_images(model, support_images, support_labels, classes=classes)

focus_embedding = []
focus_cropped_embedding = []
print("FOCUS")
for img_focus in os.listdir(focus_path):
  print(img_focus)
  img1 = os.path.join(focus_path, img_focus)
  img_embed = return_image_embedding(model_embedding, img1)
  focus_embedding.append(img_embed)

  x1, y1, x2, y2 = get_max_object(img1)
  crop_embedd = return_image_cropped_embedding(model_embedding, img1, x1, y1, x2, y2)
  focus_cropped_embedding.append(crop_embedd)

additional_embedding = []
additional_crop_embedding = []
print("ADDITIONAL FOCUS")
for img_focus in os.listdir(additional_focus_path):
  print(img_focus)
  img1_path = os.path.join(additional_focus_path, img_focus)
  img_embed = return_image_embedding(model_embedding, img1_path)
  additional_embedding.append(img_embed)

  x1, y1, x2, y2 = get_max_object(img1_path)
  crop_embedd = return_image_cropped_embedding(model_embedding, img1_path, x1, y1, x2, y2)
  additional_crop_embedding.append(crop_embedd)

all_img_name_to_embedding = {}
embedding_to_image_name = []
x_train_knn = []
y_train_knn = []
for image_name in tqdm(os.listdir(inference_image_path)):
# for image_name in ["B092SD9GKY.jpg"]:
    image_path = os.path.join(inference_image_path, image_name)
    img_embed = return_image_embedding(model_embedding, image_path)
    scr = [round(1- distance.cosine(img_embed,x), 2) for x in focus_embedding]
    if any(x >= SIMILAR_THR for x in scr):
        shutil.copy(os.path.join(inference_image_path, image_name), os.path.join(yes_path, image_name))
        include.append(image_name)
        if max(scr) >= THR_MAX_KNN:
            embedding_to_image_name.append(image_name)
            x_train_knn.append(img_embed)
            y_train_knn.append(1)
        all_img_name_to_embedding[image_name] = img_embed
    else:
        try:
            result = inference_detector(model, image_path)
        except:
            print(image_path)
            continue
        res = sorted([x for x in result[0] if x[4] >= score_thr], key=lambda x: x[4], reverse=True)
        if len(res) > 0:
            list_scr = []
            for bbox in res[0:1]:
                img = image.load_img(image_path)
                cropped = img.crop((bbox[0], bbox[1], bbox[2], bbox[3]))
                cropped.save('./test_img.png')
                # result = inference_detector(model, './test_img.png')
                # print("cropped: ", *result)
                # show_result_pyplot(model, './test_img.png', result, score_thr=score_thr)
                cropped_embed = return_image_embedding(model_embedding, cropped)
                scr = [round(1- distance.cosine(cropped_embed, x), 2) for x in additional_crop_embedding]
                list_scr.append(max(scr))
            if max(list_scr) >= SIMILAR_THR:
                shutil.copy(os.path.join(inference_image_path, image_name), os.path.join(yes_path, image_name))
                output_path = os.path.join(detect_path, image_name)
                custom_res = np.expand_dims(np.expand_dims(np.array(res[0:1][0]), 0), 0)
                model.show_result(image_path, custom_res, out_file=output_path, score_thr=score_thr)
                include.append(image_name)
                if max(list_scr) >= THR_MAX_KNN:
                    embedding_to_image_name.append(image_name)
                    x_train_knn.append(cropped_embed)
                    y_train_knn.append(1)
                all_img_name_to_embedding[image_name] = cropped_embed
            else:
                shutil.copy(os.path.join(inference_image_path, image_name), os.path.join(no_path, image_name))
                exclude.append(image_name)
                if max(list_scr) <= THR_MIN_KNN:
                    embedding_to_image_name.append(image_name)
                    x_train_knn.append(cropped_embed)
                    y_train_knn.append(0)
                all_img_name_to_embedding[image_name] = cropped_embed
        else:
            shutil.copy(os.path.join(inference_image_path, image_name), os.path.join(no_path, image_name))
            exclude.append(image_name)
            if max(scr) <= THR_MIN_KNN:
                embedding_to_image_name.append(image_name)
                x_train_knn.append(img_embed)
                y_train_knn.append(0)
            all_img_name_to_embedding[image_name] = img_embed

load checkpoint from local path: /home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/work_dirs/kettlebell_attention-rpn_5shot-fine-tuning/iter_1200.pth
cuda:0
True
FOCUS
5.jpg
4.jpg
71rZtVP7ZPL.jpg
3.jpg
2.jpg
ADDITIONAL FOCUS
B00A9HEPQY.jpg
B0B74V7Z8F.jpg
B0B4N78HKM.jpg
B0B2ZG4TRQ.jpg


100%|██████████| 5326/5326 [08:59<00:00,  9.87it/s]


In [28]:
print(len(x_train_knn), len(y_train_knn), len(embedding_to_image_name))

2653 2653 2653


In [29]:
print(len(os.listdir(yes_path)))

1744


In [30]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(x_train_knn, y_train_knn)
mat = [all_img_name_to_embedding[x] for x in os.listdir(yes_path)]
remove = []
path = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/kettlebell/fewshot_additional_focus_knn/eliminate_40_70"
for img_name in os.listdir(yes_path):
    embedd = [all_img_name_to_embedding[img_name]]
    res = neigh.predict(embedd)
    if res.item() == 0:
        shutil.copy(os.path.join(inference_image_path, img_name), os.path.join(path, img_name))
# print(len(remove))

In [31]:
print(sum(y_train_knn))

1112


In [26]:
path_high = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/kettlebell/fewshot_additional_focus_knn/higher_than_80"
path_low = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/kettlebell/fewshot_additional_focus_knn/lower_than_80"

for i in range(len(embedding_to_image_name)):
    # print(y_train_knn[i] == 1)
    if y_train_knn[i] == 1:
        shutil.copy(os.path.join(inference_image_path, embedding_to_image_name[i]), os.path.join(path_high, embedding_to_image_name[i]))
    else:
        shutil.copy(os.path.join(inference_image_path, embedding_to_image_name[i]), os.path.join(path_low, embedding_to_image_name[i]))

True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
False
True
False
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
False
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
False
True
True
False
True
True
True
True
True
False
False
True
True
False
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
False
False
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
False
True
True
True
True


### Compare results

In [14]:
print(len(os.listdir(inference_image_path)))
list_true_include = os.listdir("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/door_gt/door")
list_embedding = os.listdir("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/door/embedding_only/yes")
embedding_miss = [x for x in list_true_include if x not in list_embedding]
embedding_redundant = [x for x in list_embedding if x not in list_true_include]
miss = [x for x in list_true_include if x not in include]   
redundant = [x for x in include if x not in list_true_include]

print("Embedd MISS: ", len(embedding_miss))
print("Embedd REDUNDANT: ", len(embedding_redundant))
print("MISS: ", len(miss))
print("REDUNDANT: ", len(redundant))
# a = len(exclude)
# b = len(os.listdir("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/door_gt/not_door"))
# print("MISS: ", a - b)

2616
Embedd MISS:  344
Embedd REDUNDANT:  223
MISS:  1068
REDUNDANT:  0


In [60]:
path_1 = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/kettlebell/fewshot/detect"
path_2 = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/kettlebell/fewshot_additional_focus/detect"
diff = list(set(os.listdir(path_1)) - set(os.listdir(path_2)))
print(len(diff))
# x = [img_name for image_name in include if image_name not in embedding_miss and img_name in list_true_include]
# print(len(x))
for img_name in diff:
    # if img_name in list_true_include:
    shutil.copy(os.path.join(inference_image_path, img_name), "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/kettlebell/diff_no_add_vs_add")

80


In [21]:
diff = list(set(redundant) - set(embedding_redundant))
print(len(diff))
# x = [img_name for image_name in include if image_name not in embedding_miss and img_name in list_true_include]
# print(len(x))
for img_name in miss:
    # if img_name in list_true_include:
    shutil.copy(os.path.join(inference_image_path, img_name), "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/door/fewshot/still_missing")

42


In [13]:
path1 = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/3848/fewshot/yes"
path2 = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/3848/embedding/yes"
path_similar = "/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/3884/similar_3884"

path1_list = os.listdir(path1)
path2_list = os.listdir(path2)
path_similar_list = os.listdir(path_similar)
print(len(path1_list))
print(len(path2_list))
# print(len(path_similar_list))

diff = list(set(path1_list) - set(path2_list))
print(len(diff))
for img_name in diff:
    # if image_name not in path1_list:
    shutil.copy(os.path.join(inference_image_path, img_name),
                os.path.join("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/3848/few-embedd", img_name))

7379
7378
1


In [10]:
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

all_images = os.listdir("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/img/door")
true_list = os.listdir("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/door_gt/door")
false_list = os.listdir("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/data/door_gt/not_door")
print(len(true_list))
print(len(false_list))
y_true = [1 if img_name in true_list else 0 for img_name in all_images]

predictions = os.listdir("/home/tanluuuuuuu/Desktop/luunvt/image_retrieval/mmfewshot/result/pipeline_fewshot_retrieval/door/fewshot/yes")
y_predict = [1 if img_name in predictions else 0 for img_name in all_images]

print("Acc: ", accuracy_score(y_true, y_predict))
print("Precision: ", precision_score(y_true, y_predict))
print("Recall: ", recall_score(y_true, y_predict))
print("F1: ", f1_score(y_true, y_predict))

1061
1554
Acc:  0.7989296636085627
Precision:  0.7625122669283612
Recall:  0.7323279924599434
F1:  0.7471153846153846
